In [ ]:
from arcgis.raster.analytics import copy_raster
from os import path, listdir, mkdir
from zipfile import ZipFile
import azure.storage.blob
import requests
import datetime
import pathlib
import shutil
import arcgis
import config #local python file which is storing credentials to ArcGIS Online and Planet's Platform
import glob
import sys
import os

arcgis.env.verbose = True

In [ ]:
planet_api_key = config.planet_api_key

In [ ]:
# Provide an order ID.  
my_order_id = "INSERT ORDER ID HERE"

In [ ]:
orders_api = 'https://api.planet.com/compute/ops/orders/v2'

In [ ]:
def create_planet_session(planet_api_key):
    
    session = requests.Session()
    session.auth = (planet_api_key, "")
    
    response = session.get(orders_api)
    
    try:
        
        response = session.get(orders_api)
        
        if response.status_code == 200:
            print("Connected to Planet API")
            return(session)
        
        else:
            print("Failed to connect to Planet API")
            return
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
    

def get_order_details(session, order_id):
    
    order_url = orders_api + '/' + order_id
    
    try:
        response = session.get(order_url)
        return response.json()
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
def validate_order(order_json):

    #check if the order has been successfully completed, if not, exit the function
    if order_json['state'] != 'success':
        print("Order isn't completed yet")
        return False
    
    #also check that this is 8 band PlanetScope imagery
    if order_json["products"][0]["product_bundle"] != "analytic_8b_sr_udm2":
        print("Order is not 8 band PlanetScope Imagery")
        return False
    
    return True

def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results['_links']['results']]
    results_names = [r['name'] for r in results['_links']['results']]
    print('{} items to download'.format(len(results_urls)))
    paths=[]
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('home', 'PlanetDownloads', name))
        
        paths.append(path)
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    zip_files = [x.__str__() for x in paths if x.suffix == ".zip" ]
    
    unzipped_data_directory = pathlib.Path(os.path.join('home', 'unzipped_data'))#"/arcgis/home/unzipped_data"
    
    for zip_file in zip_files:
        z = ZipFile(zip_file)
        z.extractall(path=unzipped_data_directory)

    tiff_files = glob.glob(os.path.join(unzipped_data_directory, "**\*_SR_*.tif"), recursive = True)
    return(tiff_files)
            
def publish_to_arcgis_online(tiff_paths, order_name, gis):
    
    # Create a unique timestamp
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

    # Use timestamp and order name to create a unique name for the image service
    layer_name = "PlanetLabs_" + order_name[:8] + "_" + timestamp
    
    # Publish your images as an image service to ArcGIS Online
    # Note that this can take significant time with large datasets

    published_rasters = copy_raster(input_raster=tiff_paths,
                                    outpute_cellsize = {"distance":3.5,"units":"meters"},
                                    output_name=layer_name,
                                    raster_type_name="Raster Dataset",
                                    context={"outSR":{"wkid":3857},
                                             "resamplingMethod":"BILINEAR",
                                             "compression":"LERC 0",
                                             "bandMapping":[{"bandName":"coastal_blue","wavelengthMin":431,"wavelengthMax":452},
                                                            {"bandName":"blue","wavelengthMin":465,"wavelengthMax":515},
                                                            {"bandName":"green_i","wavelengthMin":513,"wavelengthMax":549},
                                                            {"bandName":"green","wavelengthMin":547,"wavelengthMax":583},
                                                            {"bandName":"yellow","wavelengthMin":600,"wavelengthMax":620},
                                                            {"bandName":"red","wavelengthMin":650,"wavelengthMax":680},
                                                            {"bandName":"rededge","wavelengthMin":697,"wavelengthMax":713},
                                                            {"bandName":"nir","wavelengthMin":845,"wavelengthMax":885}
                                                           ],
                                             "buildFootprints":False,
                                             "defineNodata":True,
                                             "noDataArguments":{"noDataValues":[0],
                                                                "compositeValue":True}
                                            },
                                    gis=gis)
    
    return(published_rasters)

In [ ]:
#Create a requests session with Planet API key 
planet_session = create_planet_session(planet_api_key)

#Retrieve order details as JSON
order_json = get_order_details(planet_session, my_order_id)

#Check if there are zip archives in the order delivery
zip_archives = [r['name'] for r in order_json['_links']['results'] if r['name'].endswith(".zip")]

#if the order meets criteria (8 band planetscope & a completed order)
if validate_order(order_json) is True:
    
    #if there are zips, unzip and extract
    #then return the tiff file paths
    if len(zip_archives)>0:
        download_paths = download_results(order_json)
        tiff_paths = extract_zip(download_paths)
    
    #else return the tiff paths as URLs to cloud storage
    else:
        tiff_paths = [r['location'] for r in order_json['_links']['results'] if '_SR_' in r['name']]


#publish to ArcGIS Online
gis = arcgis.gis.GIS(url="https://www.arcgis.com", username = config.arcgis_online_username, password = config.arcgis_online_password)
image_service = publish_to_arcgis_online(tiff_paths, order_json['name'], gis)

In [ ]:
# view the new imagery layer on an arcgis map
# this layer can now be added to other maps, analyzed with ArcGIS Raster Analytics tools, and more

my_map = gis.map(location = image_service.extent, zoomlevel = 11)
my_map.basemap = "imagery"
my_map.add_layer(image_service)
my_map

In [ ]:
#delete files after publishing
!rm -R /home/PlanetDownloads
!rm -R /home/unzipped_data